In [81]:
# import requests
# url = "https://drive.google.com/uc?id=1gtP1McQNNnNkQl9d4cz8aiUsnCkXWOI3&export=download"
# r = requests.get(url)
# with open('input.zip', 'wb') as f:
#    f.write(r.content)

In [82]:
# !tar -xf input.zip

In [83]:
# import requests
# url = "https://drive.google.com/uc?id=1GxuLED7FGsfa5VtI_DCWnwyP0TDrTwFe&export=download"
# r = requests.get(url)
# with open('LOC_synset_mapping.txt', 'wb') as f:
#    f.write(r.content)

In [84]:
%%writefile cam.py
import numpy as np
import cv2
import argparse

from ..configs.exploration_knowledge_config_2 import (model, gpu_transform)

from configs.exploration_knowledge_config_2 import model, gpu_transform

from torch.nn import functional as F
from torch import topk

# construct the argument parser
parser = argparse.ArgumentParser()
parser.add_argument('-i', '--input', help='path to input image', 
                    default='../data/tdt4265_2022_updated/images/val/trip007_glos_Video00003_0.png')
args = vars(parser.parse_args())

# https://github.com/zhoubolei/CAM/blob/master/pytorch_CAM.py
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

def show_cam(CAMs, width, height, orig_image, class_idx, all_classes, save_name):
    for i, cam in enumerate(CAMs):
        heatmap = cv2.applyColorMap(cv2.resize(cam,(width, height)), cv2.COLORMAP_JET)
        result = heatmap * 0.3 + orig_image * 0.5
        # put class label text on the result
        cv2.putText(result, all_classes[class_idx[i]], (20, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
        # cv2.imshow('CAM', result/255.)
        # cv2.waitKey(0)
        cv2.imwrite(f"outputs/CAM_{save_name}.jpg", result)

def load_synset_classes(file_path):
    # load the synset text file for labels
    all_classes = []
    with open(file_path, 'r') as f:
        all_lines = f.readlines()
        labels = [line.split('\n') for line in all_lines]
        for label_list in labels:
            current_class = [name.split(',') for name in label_list][0][0][10:]
            all_classes.append(current_class)
    return all_classes

# get all the classes in a list
all_classes = load_synset_classes('LOC_synset_mapping.txt')

# read and visualize the image
image = cv2.imread(args['input'])
orig_image = image.copy()
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
height, width, _ = image.shape

# load the model
model = model
# hook the feature extractor
# https://github.com/zhoubolei/CAM/blob/master/pytorch_CAM.py
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())
model._modules.get('layer4').register_forward_hook(hook_feature)
# get the softmax weight
params = list(model.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())

# define the transforms, resize => tensor => normalize
# transforms = transforms.Compose(
#     [transforms.ToPILImage(),
#      transforms.Resize((224, 224)),
#      transforms.ToTensor(),
#      transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
#     ])

transforms = gpu_transform

# apply the image transforms
image_tensor = transforms(image)
# add batch dimension
image_tensor = image_tensor.unsqueeze(0)
# forward pass through model
outputs = model(image_tensor)
# get the softmax probabilities
probs = F.softmax(outputs).data.squeeze()
# get the class indices of top k probabilities
class_idx = topk(probs, 1)[1].int()

# generate class activation mapping for the top1 prediction
CAMs = returnCAM(features_blobs[0], weight_softmax, class_idx)
# file name to save the resulting CAM image with
save_name = f"{args['input'].split('/')[-1].split('.')[0]}"
# show and save the results
show_cam(CAMs, width, height, orig_image, class_idx, all_classes, save_name)

Overwriting cam.py


In [85]:
!python cam.py --input ../data/tdt4265_2022_updated/images/val/trip007_glos_Video00003_0.png

Traceback (most recent call last):
  File "c:\Users\nikla\Documents\GitHub\TDT4265_Project\TDT4265_StarterCode\assignment4\SSD\dataset_exploration\cam.py", line 5, in <module>
    from ..configs.exploration_knowledge_config_2 import (model, gpu_transform)
ImportError: attempted relative import with no known parent package


In [86]:
!python cam.py --input ../data/tdt4265_2022_updated/images/val/trip007_glos_Video00003_77.png

Traceback (most recent call last):
  File "c:\Users\nikla\Documents\GitHub\TDT4265_Project\TDT4265_StarterCode\assignment4\SSD\dataset_exploration\cam.py", line 5, in <module>
    from ..configs.exploration_knowledge_config_2 import (model, gpu_transform)
ImportError: attempted relative import with no known parent package


In [87]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['figure.figsize'] = 15, 12

In [88]:
# image_1 = plt.imread('\outputs\CAM_image_1.jpg')
# plt.imshow(image_1)
# plt.axis('off')
# plt.show()

# image_2 = plt.imread('\outputs\CAM_image_2.jpg')
# plt.imshow(image_2)
# plt.axis('off')
# plt.show()